# Exercise 2

This notebook serves as a comprehensive solution to Exercise 2 of the VU Machine Learning course (Summer Semester 2025). The primary objective of this exercise is to deepen our understanding of Neural Networks (NNs) by implementing them using various approaches and conducting a thorough comparative analysis. All approaches are applied to the [Polish Bankruptcies Dataset](https://archive.ics.uci.edu/dataset/365/polish+companies+bankruptcy+data) as well as the [Second Dataset]().

Throughout this notebook, for each of the above mentioned datasets, we will:

- Implement a Neural Network framework from scratch: The architecture, backward and forward propagation and the entire network are built within the **nn** folder in this repo.

- Implement the same Neural Network using PyTorch: We leverage PyTorch's standard functions to create an equivalent NN, showcasing a more conventional approach to NN development.

- Utilize an LLM tool for NN implementation: Using ChatGPT 4o to generate another version of the NN from scratch, allowing for a direct comparison of code structure, design choices, and potential differences with our custom implementation.

- Investigate and experiment with NN configurations: We explore various hyperparameters, including different activation functions, numbers of layers, and nodes per layer, using a grid search approach to find optimal values.

- Analyze performance and resource usage: We calculate the total number of learnable parameters and the virtual RAM consumed by our instantiated NNs.

- Conduct a detailed comparison: The core of this notebook involves comparing the performance, efficiency, and implementation details across our custom-built NN, the PyTorch version, and the LLM-generated code. We discuss findings related to classification performance metrics and the insights gained from each implementation method.

### Setup and Imports

In [6]:
%pip install -q ucimlrepo

You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from scipy.io import arff
import requests
import io
from ucimlrepo import fetch_ucirepo

from  nn.nn import NN
from nn.layer import Layer
from nn.functions import *

## MNIST Dataset (How to use the custom NN)

In [2]:
mnist = pd.read_csv("train.csv")
split = int(len(mnist)*0.8)

y_train, y_test = mnist["label"].values[:split].astype(int), mnist["label"].values[split:].astype(int)
X_train, X_test = mnist.drop("label", axis=1).values[:split], mnist.drop("label", axis=1).values[split:]

y_train_encoded = one_hot_encoding(y_train, 10)
y_test_encoded = one_hot_encoding(y_test, 10)

In [3]:
layers = [
    Layer(input_size=X_train.shape[1], output_size=10, activation_function='softmax'),
    # Layer(input_size=10, output_size=10, activation_function='softmax'),
]

# Initialize the neural network
nn = NN(layers=layers, num_classes = 10, activation_function='softmax', loss_function='cross_entropy')

# Train the network
epochs = 5
nn.train(X_train, y_train_encoded, epochs=epochs, batch_size=100, learning_rate=0.1, verbose=True, visualize=True)

 20%|██        | 1/5 [00:02<00:09,  2.38s/it]

Epoch 1/5 ============ Loss: 0.050


 40%|████      | 2/5 [00:04<00:06,  2.32s/it]

Epoch 2/5 ============ Loss: 0.038


 60%|██████    | 3/5 [00:06<00:03,  1.99s/it]

Epoch 3/5 ============ Loss: 0.037


 80%|████████  | 4/5 [00:08<00:01,  1.97s/it]

Epoch 4/5 ============ Loss: 0.035


100%|██████████| 5/5 [00:10<00:00,  2.01s/it]

Epoch 5/5 ============ Loss: 0.034


In [4]:
nn.evaluate(X_test, y_test_encoded)

{'loss': np.float64(2.5090297915988944),
 'accuracy': np.float64(0.9091666666666667),
 'precision': np.float64(0.9085492565199148),
 'recall': np.float64(0.908163941302714),
 'f1_score': np.float64(0.9078221584260732)}

## Polish Bankruptcy Dataset

### Analytics, Visualizations and Insights

### Custom NN Modeling and Evaluation

In [8]:
train_test_split_index = int(len(X) * 0.8)
X_train = X[:train_test_split_index].values
X_test = X[train_test_split_index:].values
y_train = y[:train_test_split_index].values
y_test = y[train_test_split_index:].values
y_train_encoded = one_hot_encoding(y_train, 2)
y_test_encoded = one_hot_encoding(y_test, 2)

In [15]:
layers = [
    Layer(input_size=X_train.shape[1], output_size=10, activation_function='sigmoid'),
    Layer(input_size=10, output_size=4, activation_function='sigmoid'),
    Layer(input_size=4, output_size=2, activation_function='sigmoid'),
]

# Initialize the neural network
nn = NN(layers=layers, num_classes = 2, activation_function='sigmoid', loss_function='mean_squared_error')

# Train the network
epochs = 20
nn.train(X_train, y_train_encoded, epochs=epochs, batch_size=1000, learning_rate=0.1, verbose=True, visualize=True)

 10%|█         | 2/20 [00:00<00:02,  6.89it/s]

Epoch 1/20 ============ Loss: nan
Epoch 2/20 ============ Loss: nan


 20%|██        | 4/20 [00:00<00:02,  7.46it/s]

Epoch 3/20 ============ Loss: nan
Epoch 4/20 ============ Loss: nan


 30%|███       | 6/20 [00:00<00:02,  6.12it/s]

...

Epoch 16/20 ============ Loss: nan
Epoch 17/20 ============ Loss: nan


 95%|█████████▌| 19/20 [00:03<00:00,  5.93it/s]

Epoch 18/20 ============ Loss: nan
Epoch 19/20 ============ Loss: nan


100%|██████████| 20/20 [00:03<00:00,  6.03it/s]

Epoch 20/20 ============ Loss: nan


In [16]:
nn.evaluate(X_test, y_test_encoded)

{'loss': np.float64(nan),
 'accuracy': np.float64(0.8934454555926736),
 'precision': np.float64(0.4467227277963368),
 'recall': np.float64(0.5),
 'f1_score': np.float64(0.47186226196994585)}

### Pytorch Implementation

### LLM Developed NN Implementation

### Comparison Across the Three Implemtations

## 2nd Dataset

### Analytics, Visualization and Insights

### Custom NN Implementation

### Pytorch Implementation

### LLM Developed NN Implementation

### Comparison Across the Three Implemtations